In [70]:
import pandas as pd
from db_connect import get_db_connection

# Importer les aerports

In [71]:
airport_data = pd.read_excel('data/airports.xlsx')
airport_data = airport_data.drop(columns=['Unnamed: 0'])
airport_data

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York
...,...,...,...,...,...,...,...,...
1453,ZUN,Black Rock,35.083228,-108.791778,6454,-7,A,America/Denver
1454,ZVE,New Haven Rail Station,41.298669,-72.925992,7,-5,A,America/New_York
1455,ZWI,Wilmington Amtrak Station,39.736667,-75.551667,0,-5,A,America/New_York
1456,ZWU,Washington Union Station,38.897460,-77.006430,76,-5,A,America/New_York


# Insérez les données dans la table airport

In [72]:
data_to_insert = [
    (
        row['faa'], row['name'], row['lat'], row['lon'], row['alt'], row['tz'], row['dst'], row['tzone']
    )
    for index, row in airport_data.iterrows()
]
connection = get_db_connection()
cursor = connection.cursor()
cursor.executemany(
    "INSERT INTO airport (faa, name, lat, lon, alt, tz, dst, tzone) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
    data_to_insert
)
connection.commit()
connection.close()

Connexion réussie à la base de données SQL Server.


# Importer les vols

In [73]:
flights_data = pd.read_excel('data/flights.xlsx', header=None)
flights_data = flights_data[0].str.split(',', expand=True) # Permet de séparer les colonnes
flights_data.columns = flights_data.iloc[0] # Permet de mettre la première ligne en header
flights_data = flights_data[1:]
flights_data

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
1,2021,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2021-01-01T05:00:00Z
2,2021,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2021-01-01T05:00:00Z
3,2021,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2021-01-01T05:00:00Z
4,2021,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2021-01-01T05:00:00Z
5,2021,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2021-01-01T06:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252700,2021,7,3,1059,1100,-1,1214,1210,4,MQ,3230,N522MQ,JFK,DCA,51,213,11,0,2021-07-03T11:00:00Z
252701,2021,7,3,1059,1100,-1,1207,1213,-6,US,2171,N754UW,LGA,DCA,43,214,11,0,2021-07-03T11:00:00Z
252702,2021,7,3,1101,1110,-9,1305,1240,25,9E,3910,N813AY,JFK,RIC,61,288,11,10,2021-07-03T11:00:00Z
252703,2021,7,3,1105,1055,10,1435,1358,37,B6,971,N598JB,LGA,FLL,176,1076,10,55,2021-07-03T10:00:00Z


# Insérez les données dans la table flight

In [74]:
flights_data['time_hour'] = pd.to_datetime(flights_data['time_hour'])
data_to_insert = [
    (
         row['year'], row['month'], row['day'], row['dep_time'], row['sched_dep_time'], row['dep_delay'], row['arr_time'], row['sched_arr_time'], row['arr_delay'], row['carrier'], row['flight'], row['tailnum'], row['origin'], row['dest'], row['air_time'], row['distance'], row['hour'], row['minute'], row['time_hour']
    )
    for index, row in flights_data.iterrows()
]

# Insertion base de données
connection = get_db_connection()
cursor = connection.cursor()
cursor.executemany(
    "INSERT INTO flight (year, month, day, dep_time, sched_dep_time, dep_delay, arr_time, sched_arr_time, arr_delay, carrier, flight, tailnum, origin, dest, air_time, distance, hour, minute, time_hour)"
    "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
    data_to_insert
)
connection.commit()
connection.close()

Connexion réussie à la base de données SQL Server.
